# Intialization

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My Drive/phuc_code_file

# Import Libraries

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
from google.colab import widgets
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from utils import file_helper, feature_extraction

# Data Pipeline

In [ ]:
train_X = np.load('Dataset/split_data/Binary Split/X_train.npy')
train_y = np.load('Dataset/split_data/Binary Split/y_train.npy')
val_X = np.load('Dataset/split_data/Binary Split/X_val.npy')
val_y = np.load('Dataset/split_data/Binary Split/y_val.npy')
test_X = np.load('Dataset/split_data/Binary Split/X_test.npy')
test_y = np.load('Dataset/split_data/Binary Split/y_test.npy')

#Note:
#class       label_encoded
# Human:          1
# Non_human:      0 

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_X, val_y)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y)).batch(BATCH_SIZE)


# Build ResNet

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),

]

In [ ]:
import tensorflow.keras as keras
tf.keras.backend.clear_session()
def resnet(nb_classes): 
    n_feature_maps = 64
    input_shape = (1,5704)
    input_layer = keras.layers.Input(input_shape)

    #BLOCK 1 
    conv_x = keras.layers.Conv1D(filters = n_feature_maps, kernel_size = 7, padding = 'same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters = n_feature_maps, kernel_size = 5, padding = 'same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters = n_feature_maps, kernel_size = 3, padding = 'same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    #expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters = n_feature_maps, kernel_size = 1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    #BLOCK 2
    conv_x = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 7, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 5, padding = 'same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    #Expand Channels for sum
    shortcut_y = keras.layers.Conv1D(filters = n_feature_maps *2, kernel_size = 1, padding = 'same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    #BLOCK 3
    conv_x = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 7, padding = 'same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 5, padding = 'same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters = n_feature_maps * 2, kernel_size = 3, padding = 'same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    #No need to expand channels anmr, cause now they are equal 
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    #FINAL 

    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

    output_layer = keras.layers.Dense(nb_classes, activation='sigmoid')(gap_layer)

    return keras.models.Model(inputs = input_layer, outputs = output_layer)

In [ ]:
model = resnet(1)
model.summary()

# Optimizer, Callbacks, Reduce learning rates, early stopping

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience = 5, verbose = 1)
# RMSprop(learning_rate= 1e-5)
model.compile(optimizer = RMSprop(learning_rate = 1e-2), loss = tf.keras.losses.binary_crossentropy, metrics =METRICS)

In [ ]:
import os 
#Provides unique names for checkpoints and adjust the checkpointing frequency
checkpoint_path = "ResNet_Binary/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a call back that save the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 save_best_only=True,
                                                 verbose = 1,
                                                 period = 10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15, verbose= 1, min_lr=1e-9)



# Train Model

In [ ]:
model_train = model.fit(train_dataset,
                        epochs = 100,
                        validation_data = val_dataset,
                        batch_size=128,
                        callbacks= [cp_callback, reduce_lr]
                        )#,callbacks = [lr_schedule]callbacks = [early_stop],callbacks = [cp_callback]

# Plot Loss vs accuracy

In [ ]:
plt.figure(figsize =(15,4))
plt.plot(model_train.epoch, model_train.history["loss"], label="Train")
plt.plot(model_train.epoch, model_train.history["val_loss"], label="Valid")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize =(15,4))
plt.plot(model_train.epoch, model_train.history["accuracy"], label="Train")
plt.plot(model_train.epoch, model_train.history["val_accuracy"], label="Valid")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
test_eval = model.predict(test_X, batch_size = 64)
model.evaluate(test_dataset, batch_size= 64, verbose = 1)

# Test Model

In [ ]:
from sklearn.metrics import classification_report
pred = np.round(test_eval, 0)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
label = ["NON_HUMAN","HUMAN"]
target = ["Class {}".format(i) for i in range(5)]
classification_metrics = classification_report(test_y ,pred, target_names = label)
print(classification_metrics)

In [ ]:
model.save("ResNet_Binary.h5")

In [ ]:
from utils import confusion_matrix_pretty_print
from utils.confusion_matrix_pretty_print import plot_confusion_matrix_from_data

columns = ["NON_HUMAN","HUMAN"]
annot = True
cmap = 'Blues'
fmt = '.2f'
lw = 0.5
cbar = False
show_null_values = 2
pred_val_axis = 'y'
#size::
fz = 12
figsize = [9,9]
if(len(test_y) > 10):
  fz=9; figsize=[14,14]
plot_confusion_matrix_from_data(test_y, pred, columns,
      annot, cmap, fmt, fz, lw, cbar, figsize, show_null_values, pred_val_axis)